# Test SR model

In [7]:
import numpy as np 
import torch
from PIL import Image
import matplotlib.pyplot as plt

# 假设你的项目路径配置正确
from src.emcfsys.EMCellFiner.hat.models.hat_model import HATModel
from src.emcfsys.EMCellFiner.hat.models.img_utils import tensor2img

# 1. 初始化模型
path = r"D:\napari_EMCF\EMCFsys\models\EMCellFiner.pth"
# 显式指定 tile_size，防止显存溢出；对于小图可以不用 tile
model = HATModel(path, scale=4, tile_size=512) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. 读取与预处理
img_path = r"D:\napari_EMCF\EMCFsys\emcfsys\image\Bock2011_2951_XrV1ciGgTWHjepNf.tif"
# [修正1] crop 使用元组 (left, top, right, bottom)
# 这里的坐标代表: x从0到128(宽), y从0到512(高)
img = Image.open(img_path).convert("RGB")

# 转换为 Numpy 并归一化
img_np = np.array(img).astype(np.float32) / 255.

# 转换为 Tensor: (H, W, C) -> (C, H, W) -> (1, C, H, W)
img_torch = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).to(device)

print(f"Input Shape: {img_torch.shape}")

# 3. 推理
# 之前定义的 HATModel.forward 默认 use_tile=True
# 对于 128x512 这种小图，其实可以直接 use_tile=False 速度更快
with torch.no_grad():
    output = model(img_torch) 

print(f"Output Shape: {output.shape}")

# 4. 后处理
output = output.cpu()

# [修正2] 关键！设置 rgb2bgr=False
# 因为 Image.fromarray 需要 RGB 格式，而 tensor2img 默认转为 BGR (给OpenCV用的)
img_out = tensor2img(output, rgb2bgr=False, min_max=(0, 1))

# 5. 转回 PIL 图片
img_final = Image.fromarray(img_out)

# 验证结果
# img_final.show() 
# img_final.save("result_sr.png")
print("Done.")

Dwonloading model from : https://github.com/yzy0102/emcfsys/releases/latest/download/EMCellFiner.pth
Input Shape: torch.Size([1, 3, 1024, 1024])
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Output Shape: torch.Size([1, 3, 4096, 4096])
Done.
